In [1]:
import tensorflow as tf
import numpy as np

2024-08-09 21:33:37.388528: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 21:33:37.424923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 21:33:37.424960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 21:33:37.425933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-09 21:33:37.431660: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 21:33:37.432090: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
print('TF Version:', tf.__version__)
# Should work with TF 2.15.0

TF Version: 2.15.0


## MCTS Chess Neural Network
---

[notes go here]

In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

class SimpleChessNet(Model):
    
    def __init__(self, action_size,
                 board_input_shape=(8,8,6),
                 n_conv_channels=12,
                 dropout_rate=0.5,
                 pi_weight=300.0):
        
        super(SimpleChessNet, self).__init__()
        
        self.board_x, self.board_y, self.board_channels = board_input_shape
        self.action_size = action_size
        self.pi_weight = tf.constant(pi_weight)
        
        self.input_layer = Input(shape=board_input_shape)
        
        
        h_conv1 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='same', use_bias=False)(self.input_layer))) # batch_size  x board_x x board_y x num_channels
        h_conv2 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='same', use_bias=False)(h_conv1)))          # batch_size  x board_x x board_y x num_channels
        h_conv3 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='valid', use_bias=False)(h_conv2)))         # batch_size  x (board_x-2) x (board_y-2) x num_channels
        h_conv4 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='valid', use_bias=False)(h_conv3)))         # batch_size  x (board_x-4) x (board_y-4) x num_channels
        h_conv4_flat = Flatten()(h_conv4)
        
        s_fc1 = Dropout(dropout_rate)(Activation('elu')(BatchNormalization(axis=1)(Dense(1024, use_bias=False)(h_conv4_flat))))  # batch_size x 1024
        s_fc2 = Dropout(dropout_rate)(Activation('elu')(BatchNormalization(axis=1)(Dense(512, use_bias=False)(s_fc1))))          # batch_size x 1024
        
        # get the policy and value estimates:
        self.pi = Dense(self.action_size, activation='softmax', name='pi')(s_fc2) # batch_size x self.action_size
        self.v = Dense(1, activation='tanh', name='v')(s_fc2)
        
        self.model = Model(inputs=self.input_layer, outputs=[self.pi, self.v])
        # self.model.compile(loss=['categorical_crossentropy', 'mean_squared_error'], optimizer=Adam(learning_rate))
        
    def call(self, x):
        return self.model(x)
        
    def __call__(self, x):
        return self.model(x)
        
    def compute_loss(self, x, pi, v):
        policy_pred, v_pred = self.__call__(x)
        v_loss = tf.reduce_mean((v_pred - v)**2)
        pi_loss = tf.reduce_mean(-pi*tf.math.log(policy_pred))
        total_loss = v_loss + self.pi_weight*pi_loss #tf.add_n([(v_loss + pi_loss)] + self.losses)
        return v_loss, pi_loss, total_loss
    
    def save(self, path):
        tf.saved_model.save(self.model, path)


In [4]:
nn = SimpleChessNet(action_size=64*64)
nn.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 8, 8, 6)]            0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 8, 8, 12)             648       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 8, 8, 12)             48        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 8, 8, 12)             0         ['batch_normalization[0][0

In [5]:
from tensorflow.keras.optimizers import *

class ChessNetModule(tf.Module):
    
    def __init__(self, learning_rate=1e-5):
        super(ChessNetModule, self).__init__()
        self.model = SimpleChessNet(action_size=64*64)
        self.optimizer = Adam(learning_rate)
        self.checkpoint = tf.train.Checkpoint(self.model)
        
        
        
    @tf.function(input_signature=[tf.TensorSpec([None,8,8,6])])
    def __call__(self, x):
        return self.model(x)
    
    @tf.function(input_signature=[tf.TensorSpec([None,8,8,6], tf.float32), 
                                  tf.TensorSpec([None,64*64], tf.float32), 
                                  tf.TensorSpec([None], tf.float32)])
    def train_step(self, x, pi, v):
        with tf.GradientTape() as tape:
            v_loss, pi_loss, total_loss = self.model.compute_loss(x, pi, v)
        
        grads = tape.gradient(total_loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
        
        return v_loss, pi_loss, total_loss
    
    @tf.function(input_signature=[tf.TensorSpec([None,8,8,6], tf.float32), 
                                  tf.TensorSpec([None,64*64], tf.float32), 
                                  tf.TensorSpec([None], tf.float32)])
    def validate_step(self, x, pi, v):
            v_loss, pi_loss, total_loss = self.model.compute_loss(x, pi, v)
            return v_loss, pi_loss, total_loss
    
    @tf.function(input_signature=[tf.TensorSpec([], tf.string)])
    def save_checkpoint(self, path):
        self.checkpoint.write(path)
        return path
    
    @tf.function(input_signature=[tf.TensorSpec([], tf.string)])
    def reset_optimizer(self, options):
        for var in self.optimizer.variables:
            var.assign(tf.zeros_like(var))

        return options
    
    def save(self, path):
        tf.saved_model.save(self, path, 
        signatures={
            'chessnet_serve' : 
                self.__call__.get_concrete_function(tf.TensorSpec([None,8,8,6], tf.float32)),
            'chessnet_train' : 
                self.train_step.get_concrete_function(tf.TensorSpec([None,8,8,6], tf.float32),
                                                      tf.TensorSpec([None,64*64], tf.float32),
                                                      tf.TensorSpec([None], tf.float32)),
            'chessnet_validate' : 
                self.validate_step.get_concrete_function(tf.TensorSpec([None,8,8,6], tf.float32),
                                                      tf.TensorSpec([None,64*64], tf.float32),
                                                      tf.TensorSpec([None], tf.float32)),
            'chessnet_save_checkpoint':
               self.save_checkpoint.get_concrete_function(tf.TensorSpec([], tf.string)),
            'chessnet_reset_optimizer':
               self.reset_optimizer.get_concrete_function(tf.TensorSpec([], tf.string)),
        })

In [6]:
nn_mod = ChessNetModule()

x_test = tf.random.uniform(shape=[1,8,8,6])
print(nn_mod(x_test))


[<tf.Tensor: shape=(1, 4096), dtype=float32, numpy=
array([[0.00023931, 0.00025386, 0.00024758, ..., 0.00023651, 0.00027289,
        0.00023098]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.18545932]], dtype=float32)>]


In [7]:
nn_mod.save('simple_chess_net_v3')

INFO:tensorflow:Assets written to: simple_chess_net_v3/assets


INFO:tensorflow:Assets written to: simple_chess_net_v3/assets
